In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import  *
from pyspark.sql.window import Window

### data reading


In [0]:
df= spark.read.format("parquet").load("abfss://bronze@eteproject.dfs.core.windows.net/orders")

In [0]:
display(df)


In [0]:
dropdf=df.drop('_rescued_data')
display(dropdf)

In [0]:

df=dropdf.withColumn("order_date",to_timestamp(col('order_date')))
display(df)

In [0]:
df= df.withColumn("year",year(col('order_date')))
display(df)


In [0]:
df1 = df.withColumn("flag", dense_rank().over(Window.partitionBy("year").orderBy(desc("total_amount"))))

In [0]:
df1.display()

In [0]:
df1 = df1.withColumn("row_flag", row_number().over(Window.partitionBy("year").orderBy(desc("total_amount"))))
display(df1)

### oop in pyspark - class

In [0]:
class windows: 
    def dense_rank (self,df):
     df_dense_rank =df.withColumn("flag", dense_rank().over(Window.partitionBy("year").orderBy(desc("total_amount"))))
     return df_dense_rank
    
    def rank (self,df):
     df_rank =df.withColumn("rank_flag", rank().over(Window.partitionBy("year").orderBy(desc("total_amount"))))
     return df_rank
    
    def row(self,df):
     df_row =df.withColumn("row_flag", row_number().over(Window.partitionBy("year").orderBy(desc("total_amount"))))
     return df_row

In [0]:
df_new= df

In [0]:
obj= windows()


In [0]:
df_results=obj.dense_rank(df_new)


### data writing


In [0]:
df.write.format("delta").mode("overwrite").save("abfss://silver@eteproject.dfs.core.windows.net/orders")

In [0]:
%sql 
create table if not exists databricks_ete.silver.orders_silver
using delta
location "abfss://silver@eteproject.dfs.core.windows.net/orders"
